In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import geopandas
import descartes

In [2]:
path = "/home/administrator/Documents/Web-based-data/Project/MobilePhoneData/"
SET3_M01 = pd.read_csv(path + "SET3/SET3_M01.CSV", header = None, names = ["user_id", "timestamp", "arrondissement_id"])

In [3]:
most_active_users = SET3_M01.user_id.value_counts()[:100]
most_active_users.index

Int64Index([ 29741, 115669, 145890, 145762, 101780,  92583,  28765,  87604,
            101855, 107792, 127730,  92240,  13891,  86115,  48980, 101701,
             91665,  87173,  25832,  78849,  41280,  81833, 148777,  33108,
            115364, 127341, 129963,  27595,  71657,  45993,  29689, 128839,
            111876,  43849, 112159,  17970,  12429,  68908,  74900,   6037,
            124270,  67670,  35618,  87609,  88454,  77536,  26169, 115710,
            119082,  57297, 122007,  51935,  94584, 139337,  79808,  84043,
             85262, 117044,  12960,  75593, 117855, 114672, 127979, 123940,
             56885,   8969,   1325, 100780,  96290,  50040, 103869,  50323,
             10572,  87460,  31722,  17247,   5898,  45608,  70530, 133630,
            127150,  26662,  79964, 117018,  58676, 124386, 116262,   6011,
             25782,  96096,  52888,  28202,  26199, 115924,  92697,  37231,
             65478,  47767, 136231,  13342],
           dtype='int64')

In [5]:
SET3_M01_most_active = SET3_M01[SET3_M01['user_id'].isin(most_active_users.index)]

In [38]:
SET3_M01_most_active['timestamp'] = pd.to_datetime(SET3_M01_most_active['timestamp'])
SET3_M01_most_active['date'] = SET3_M01_most_active['timestamp'].dt.normalize()

/home/administrator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/administrator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [41]:
pref_arr_most_active = SET3_M01_most_active.groupby(['user_id', 'date'])['arrondissement_id'].agg(lambda x: pd.Series.mode(x)[0])

In [42]:
pref_arr_most_active.head()

user_id  date      
1325     2013-01-01    4
         2013-01-02    4
         2013-01-03    4
         2013-01-04    4
         2013-01-05    4
Name: arrondissement_id, dtype: int64

In [43]:
pref_arr_most_active.to_csv("/home/administrator/Documents/Web-based-data/Project/MobilePhoneData/SET3/MO1_pref_arr_most_active.csv")

/home/administrator/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [114]:
pref_arr_most_active = pd.read_csv("/home/administrator/Documents/Web-based-data/Project/MobilePhoneData/SET3/MO1_pref_arr_most_active.csv",
                                  header = None, names = ["user_id", "date", "pref_arr"])

In [134]:
pref_arr_most_active.head()

,user_id,date,pref_arr,end_arr
0,1325,2013-01-01,4,4
1,1325,2013-01-02,4,4
2,1325,2013-01-03,4,4
3,1325,2013-01-04,4,4
4,1325,2013-01-05,4,4


In [47]:
#pref_arr_most_active_wide = pref_arr_most_active.pivot(index='user_id', columns='date', values='pref_arr')

In [62]:
#pref_arr_most_active_wide.head()

In [115]:
pref_arr_most_active['end_arr'] = 0 

In [128]:
for index, row in pref_arr_most_active.iterrows():
    
    if row['date'] == '2013-01-31':
        value = row['pref_arr']
        pref_arr_most_active.iloc[index, pref_arr_most_active.columns.get_loc('end_arr')] = value
        pref_arr_most_active.iloc[index-1, pref_arr_most_active.columns.get_loc('end_arr')] = value
    
    else:
        value = row['pref_arr']
        pref_arr_most_active.iloc[index-1, pref_arr_most_active.columns.get_loc('end_arr')] = value 
    
        

In [130]:
start_end_most_active = pref_arr_most_active.copy()

In [131]:
start_end_most_active = start_end_most_active.rename(columns = {'pref_arr': 'start_arr'})

In [133]:
start_end_most_active.to_csv("/home/administrator/Documents/Web-based-data/Project/MobilePhoneData/SET3/MO1_start_end_most_active.csv")

In [139]:
start_end_count = start_end_most_active.groupby(['start_arr', 'end_arr']).count()

In [141]:
start_end_count = start_end_count.drop('date', 1)
start_end_count = start_end_count.rename(columns = {'user_id': 'count'})

In [142]:
start_end_count.head()

count
start_arr end_arr       
1         1          410
          2           24
          3           25
          4           15
          5            5

In [143]:
start_end_count.to_csv("/home/administrator/Documents/Web-based-data/Project/MobilePhoneData/SET3/MO1_start_end_count.csv")